In [4]:
import json
from os import environ
from dotenv import load_dotenv
import pandas as pd
import polars as pl
import requests
load_dotenv("../")

API_KEY = environ["RIOT_API"]

In [5]:
class Region:
    def __init__(self) -> str:
        self.na = "na1"
        self.br = "br1"
        self.eun = "eun1"
        self.euw = "euw1"
        self.jp = "jp1"
        self.kr = "kr"
        self.la1 = "la1"
        self.la2 = "la2"
        self.oc = "oc1"
        self.ph = "ph2"
        self.ru = "ru"
        self.sg = "sg2"
        self.th = "th2"
        self.tr = "tr1"
        self.tw = "tw2"
        self.vn = "vn2"

In [19]:
class Summoner:
    def __init__(self, region, summoner_name):
        self.data = self.get_data(region, summoner_name)
        self.id = self.data.id[0]
        self.account_id = self.data.accountId[0]
        self.puuid = self.data.puuid[0]
        self.name = self.data.name[0]
        self.profile_icon_id = self.data.profileIconId[0]
        self.revision_date = self.data.revisionDate[0]
        self.summoner_level = self.data.summonerLevel[0]
        self.masteries = self.get_mastery_data(region, self.id)
        
    def get_data(self, region, summoner_name):
        summoner_url = f"https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}" + "?api_key=" + API_KEY
        req = requests.get(summoner_url)
        data = pd.json_normalize(req.json())
        
        return data
        
    def get_mastery_data(self, region, id):
        mastery_url = f"https://{region}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{id}" + "?api_key=" + API_KEY
        req = requests.get(mastery_url)
        data = pd.DataFrame.from_dict(req.json())

        return data



In [20]:
yogurtsauce = Summoner(Region().na, "yogurtsauce")
yogurtsauce.masteries

,puuid,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,chestGranted,tokensEarned,summonerId
0,9e22e09b-2d08-54dc-97e7-fad7d9adb82c,11,5,144341,1682807457000,122741,0,False,1,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
1,NaN,67,7,123733,1684566786000,102133,0,True,0,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
2,9e22e09b-2d08-54dc-97e7-fad7d9adb82c,107,7,107245,1684670703000,85645,0,True,0,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
3,NaN,412,6,106448,1679258580000,84848,0,False,1,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
4,NaN,81,5,79674,1684120407000,58074,0,False,1,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
...,...,...,...,...,...,...,...,...,...,...
158,NaN,711,2,1966,1653189439000,166,4034,False,0,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
159,NaN,888,1,1611,1666768012000,1611,189,False,0,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
160,NaN,902,1,1196,1683366972000,1196,604,False,0,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
161,NaN,897,1,1032,1678052551000,1032,768,False,0,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q


In [18]:
test = f"https://na1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q"
test = test + "?api_key=" + API_KEY

test2 = requests.get(test)
pd.DataFrame.from_dict(test2.json())

,puuid,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,chestGranted,tokensEarned,summonerId
0,9e22e09b-2d08-54dc-97e7-fad7d9adb82c,11,5,144341,1682807457000,122741,0,False,1,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
1,NaN,67,7,123733,1684566786000,102133,0,True,0,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
2,9e22e09b-2d08-54dc-97e7-fad7d9adb82c,107,7,107245,1684670703000,85645,0,True,0,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
3,NaN,412,6,106448,1679258580000,84848,0,False,1,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
4,NaN,81,5,79674,1684120407000,58074,0,False,1,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
...,...,...,...,...,...,...,...,...,...,...
158,NaN,711,2,1966,1653189439000,166,4034,False,0,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
159,NaN,888,1,1611,1666768012000,1611,189,False,0,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
160,NaN,902,1,1196,1683366972000,1196,604,False,0,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
161,NaN,897,1,1032,1678052551000,1032,768,False,0,U8TV9fzVGKAowHPEnCZt2CMAp4_mgDtYirzNGEiAXJoZn4Q
